## Instruction

> 1. Rename Assignment-01-###.ipynb where ### is your student ID.
> 2. The deadline of Assignment-01 is 23:59pm, 03-31-2024
>
> 3. In this assignment, you will
>    1) explore Wikipedia text data
>    2) build language models
>    3) build NB and LR classifiers

## Task0 - Download datasets
> Download the preprocessed data, enwiki-train.json and enwiki-test.json from the Assignment-01 folder. In the data file, each line contains a Wikipedia page with attributes, title, label, and text. There are 1000 records in the train file and 100 records in test file with ten categories.

## Task1 - Data exploring and preprocessing

> 1) Print out how many documents are in each class  (for both train and test dataset)

In [11]:
# Your code goes to here
import json
from pprint import pprint
test_data = []
train_data = []
with open('enwiki-test.json', encoding='utf-8') as f:
    for line in f:
        test_data.append(json.loads(line))
with open('enwiki-train.json', encoding='utf-8') as f:
    for line in f:
        train_data.append(json.loads(line))
print(type(test_data[0]),len(test_data))
print(type(train_data[0]),len(train_data))
pprint(train_data[0].keys())
pprint(train_data[0])




<class 'dict'> 100
<class 'dict'> 1000
dict_keys(['title', 'label', 'text'])
{'label': 'Film',
 'text': 'Citizen Kane is a 1941 American drama film produced by, directed by, '
         'and starring Orson Welles. He also co-wrote the screenplay with '
         "Herman J. Mankiewicz. The picture was Welles' first feature film. "
         '"Citizen Kane" is considered by many critics and experts to be the '
         'greatest film ever made. For 50 consecutive years, it stood at '
         'number 1 in the British Film Institute\'s "Sight &amp; Sound" '
         'decennial poll of critics, and it topped the American Film '
         "Institute's 100 Years\xa0... 100 Movies list in 1998, as well as its "
         '2007 update. The film was nominated for Academy Awards in nine '
         'categories and it won for Best Writing (Original Screenplay) by '
         'Mankiewicz and Welles. "Citizen Kane" is praised for Gregg Toland\'s '
         "cinematography, Robert Wise's editing, Bernard H

> 2) Print out the average number of sentences in each class.
>    You may need to use sentence tokenization of NLTK.
>    (for both train and test dataset)


In [13]:
# Your code goes to here
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to D:\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [16]:
segmented_train = []
segmented_test = []
for article in train_data:
    segmented_train.extend(sent_tokenize(article['text']))
for article in test_data:
    segmented_test.extend(sent_tokenize(article['text']))


338226
24440


In [22]:
print(f"There are {len(segmented_train)/len(train_data)} sentences in train dataset on average")
print(f"There are {len(segmented_test)/len(test_data)} sentences in test dataset on average")
# print(segmented_train[0])

There are 338.226 sentences in train dataset on average
There are 244.4 sentences in test dataset on average


> 3) Print out the average number of tokens in each class
>    (for both train and test dataset)

In [23]:
# Your code goes to here
import spacy
import tqdm
nlp = spacy.load('en_core_web_sm')
def spacy_tokenize(text):
    return [token.text for token in nlp(text)]
train_token = []
test_token = []
for sent in segmented_train:
    train_token.extend(spacy_tokenize(sent))
for sent in segmented_test:
    test_token.extend(spacy_tokenize(sent))

In [26]:
from nltk.tokenize import word_tokenize
train_token_nltk = []
test_token_nltk = []
for article in train_data:
    train_token_nltk.extend(word_tokenize(article['text']))
for article in test_data:
    test_token_nltk.extend(word_tokenize(article['text']))
print(f"Train data has {len(train_token_nltk)/len(train_data)} tokens on average")
print(f"Test data has {len(test_token_nltk)/len(test_data)} tokens on average")



Train data has 8695.333 tokens on average
Test data has 5222.63 tokens on average


In [24]:
print(f"There are {len(train_token)/len(train_data)} tokens in train dataset on average")
print(f"There are {len(test_token)/len(test_data)} tokens in test dataset on average")

There are 8852.463 tokens in train dataset on average
There are 6253.16 tokens in test dataset on average


> 4) For each sentence in the document, remove punctuations and other special characters so that each sentence only contains English words and numbers. To make your life easier, you can make all words as lower cases. For each class, print out the first article's name and the processed first 40 words. (for both train and test dataset)

In [37]:
# Your code goes to here
import re
def sentece_preprocess(document:list):
    for article in document:
        article['text'] = re.sub(r'[^a-zA-Z0-9\s]', '',  article['text']).lower()
    return document

punctuation_removed_test = sentece_preprocess(test_data)
print(punctuation_removed_test)
for article in punctuation_removed_test:
    print(article['title'])
    print(article['text'][:40])
    break



[{'title': "Monty_Python's_Life_of_Brian", 'label': 'Film', 'text': 'monty pythons life of brian also known as life of brian is a 1979 british comedy film starring and written by the comedy group monty python graham chapman john cleese terry gilliam eric idle terry jones and michael palin it was directed by jones the film tells the story of brian cohen played by chapman a young jewishroman man who is born on the same day asand next door tojesus and is subsequently mistaken for the messiah\nfollowing the withdrawal of funding by emi films just days before production was scheduled to begin longtime monty python fan and former member of the beatles george harrison arranged financing for life of brian through the formation of his company handmade films\nthe films themes of religious satire were controversial at the time of its release drawing accusations of blasphemy and protests from some religious groups thirtynine local authorities in the united kingdom either imposed an outright ban or

## Task2 - Build language models

> 1) Based on the training dataset, build unigram, bigram, and trigram language models using Add-one smoothing technique. It is encouraged to implement models by yourself. If you use public code, please cite it.


In [38]:
from nltk.tokenize import word_tokenize
from collections import Counter, defaultdict

In [42]:
# Example training dataset

# Tokenize texts
tokenized_train_texts = [word_tokenize(doc['text'].lower()) for doc in train_data]


def build_ngram_model(tokenized_texts, n):
    all_tokens = [token for sublist in tokenized_texts for token in sublist]
    total_tokens = len(all_tokens)
    
    if n > 1:
        all_tokens = ['<s>']*(n-1) + all_tokens + ['</s>']
        total_tokens += n-1  
    
    # Count n-grams and (n-1)-grams
    ngrams = Counter(zip(*[all_tokens[i:] for i in range(n)]))
    if n == 1:
        n_minus_one_grams = {(): sum(ngrams.values())}  
    else:
        n_minus_one_grams = Counter(zip(*[all_tokens[i:] for i in range(n-1)]))
    
    # Calculate probabilities with Add-one smoothing
    vocabulary_size = len(set(all_tokens))
    model = defaultdict(lambda: 1/(total_tokens + vocabulary_size))  # Default probability
    
    for ngram in ngrams:
        previous_ngram = ngram[:-1] if n > 1 else ()
        model[ngram] = (ngrams[ngram] + 1) / (n_minus_one_grams[previous_ngram] + vocabulary_size)
    
    return model

# Build n-gram models
unigram_model = build_ngram_model(tokenized_train_texts, 1)
bigram_model = build_ngram_model(tokenized_train_texts, 2)
trigram_model = build_ngram_model(tokenized_train_texts, 3)

# Example: print some probabilities
print(unigram_model[('this',)])
print(bigram_model[('<s>', 'this')])
print(trigram_model[('<s>', '<s>', 'this')])


0.0024878869389715658
1.1283441864999486e-07
1.1283440591839026e-07


> 2) Report the perplexity of these 3 trained models on the testing dataset and explain your findings. 

In [43]:
# Your code goes to here
from math import log2

def calculate_perplexity(model, tokenized_text, n):
    log_probability = 0
    N = 0
    for i in range(len(tokenized_text) - n + 1):
        if n == 1:
            ngram = (tokenized_text[i],)
        else:
            ngram = tuple(tokenized_text[i:i+n])
        log_probability += log2(model[ngram])
        N += 1
    return 2 ** (-log_probability / N)

def pad_text(tokenized_text, n):
    if n == 1:
        return tokenized_text  # No padding needed for unigram model
    else:
        return [['<s>']*(n-1) + text + ['</s>'] for text in tokenized_text]

tokenized_test_texts = [word_tokenize(doc['text'].lower()) for doc in test_data]

# Compute perplexity for each model
for n, model in zip([1, 2, 3], [unigram_model, bigram_model, trigram_model]):
    padded_test_texts = pad_text(tokenized_test_texts, n)
    perplexities = [calculate_perplexity(model, text, n) for text in padded_test_texts]
    avg_perplexity = sum(perplexities) / len(perplexities)
    print(f"Average perplexity for the {n}-gram model: {avg_perplexity}")



Average perplexity for the 1-gram model: 3705.7382988357062
Average perplexity for the 2-gram model: 56672.45469827124
Average perplexity for the 3-gram model: 1545080.708897


> 3) Use each built model to generate five sentences and explain these generated patterns.


In [48]:
# Your code goes to here
import random

def generate_sentence_from_unigram(unigram_model, sentence_length=20):
    sentence = []
    for _ in range(sentence_length):
        # Choose the next word based on unigram probabilities
        word = random.choices(list(unigram_model.keys()), weights=unigram_model.values())[0][0]
        sentence.append(word)
        if word == '</s>':  # Stop if the end of sentence token is generated
            break
    return ' '.join(sentence)

def generate_sentence_from_bigram(model, start_token='<s>', end_token='</s>', max_length=20):
    sentence = [start_token]
    while len(sentence) < max_length:
        prev_word = sentence[-1]
        candidates = [(word, prob) for ((first_word, word), prob) in model.items() if first_word == prev_word]
        if not candidates:  # If no candidates are available, stop.
            break
        words, probabilities = zip(*candidates)
        next_word = random.choices(words, weights=probabilities)[0]
        if next_word == end_token:
            break
        sentence.append(next_word)
    return ' '.join(sentence[1:])  # Remove start token from final sentence

def generate_sentence_from_trigram(model, start_tokens=('<s>', '<s>'), end_token='</s>', max_length=20):
    sentence = list(start_tokens)
    while len(sentence) < max_length + len(start_tokens):  # Adjust for start tokens
        prev_words = tuple(sentence[-2:])  # Last two words
        candidates = [(word, prob) for ((first_word, second_word, word), prob) in model.items() if first_word == prev_words[0] and second_word == prev_words[1]]
        if not candidates:  # If no candidates are available, stop.
            break
        words, probabilities = zip(*candidates)
        next_word = random.choices(words, weights=probabilities)[0]
        if next_word == end_token:
            break
        sentence.append(next_word)
    return ' '.join(sentence[2:])  # Remove start tokens from final sentence


# Example usage (Assuming `unigram_model` is already built)
unigram_generated_sentences = generate_sentence_from_unigram(unigram_model)
bigram_generated_sentences = generate_sentence_from_bigram(bigram_model)
trigram_generated_sentences = generate_sentence_from_trigram(trigram_model)
# print(bigram_generated_sentences)
print(trigram_generated_sentences)



citizen kane '' , italy . the pressure has been praised sargent began planning to run because lindelof said
gustave moreau , characterized zhou as an impostor , can lead to the city through an echo of the and


## Task3 - Build NB/LR classifiers

> 1) Build a Naive Bayes classifier (with Laplace smoothing) and test your model on test dataset

In [50]:
# Example dataset
documents = [
    ('Chinese Beijing Chinese', 'yes'),
    ('Chinese Chinese Shanghai', 'yes'),
    ('Chinese Macao', 'yes'),
    ('Tokyo Japan Chinese', 'no')
]
test_documents = [
    ('Chinese Chinese Chinese Tokyo Japan', '?')
]


In [56]:
from collections import defaultdict
import numpy as np

from collections import defaultdict

def create_vocabulary_and_counts(documents):
    vocabulary = set()
    word_counts_per_class = defaultdict(lambda: defaultdict(int))
    class_counts = defaultdict(int)
    
    for doc in documents:
        text = doc['text']  # Use the 'text' field
        label = doc['label']  # The label remains accessed the same way
        words = text.split()  # Tokenize the text
        vocabulary.update(words)
        class_counts[label] += 1
        for word in words:
            word_counts_per_class[label][word] += 1
            
    return vocabulary, word_counts_per_class, class_counts

vocabulary, word_counts_per_class, class_counts = create_vocabulary_and_counts(train_data)

In [57]:
class NaiveBayesClassifier:
    def __init__(self, vocabulary, word_counts_per_class, class_counts, alpha=1):
        self.vocabulary = vocabulary
        self.word_counts_per_class = word_counts_per_class
        self.class_counts = class_counts
        self.alpha = alpha  # Laplace smoothing factor
        self.num_docs = sum(class_counts.values())
        self.num_words_per_class = {label: sum(word_counts.values()) for label, word_counts in word_counts_per_class.items()}
    
    def predict(self, document):
        words = document.split()
        class_probabilities = {}
        
        for label in self.class_counts:
            log_prob = np.log(self.class_counts[label] / self.num_docs)
            total_word_count = self.num_words_per_class[label]
            
            for word in words:
                word_count = self.word_counts_per_class[label].get(word, 0)
                # Calculate log probabilities to avoid underflow
                log_prob += np.log((word_count + self.alpha) / (total_word_count + self.alpha * len(self.vocabulary)))
                
            class_probabilities[label] = log_prob
            
        return max(class_probabilities, key=class_probabilities.get)

In [61]:
# Instantiate and test the Naive Bayes classifier
def calculate_accuracy(classifier, test_data):
    correct_predictions = 0
    total_predictions = len(test_data)
    
    for doc in test_data:
        text = doc['text']
        actual_label = doc['label']
        predicted_class = classifier.predict(text)
        
        if predicted_class == actual_label:
            correct_predictions += 1
            
        print(f"Document sample is predicted as '{predicted_class}' (Actual: '{actual_label}')")
    
    accuracy = correct_predictions / total_predictions
    return accuracy

# Assuming test_data is structured similarly to train_data
# test_data = [{'title': 'Sample title 5', 'text': 'Some test text', 'label': 'yes/no'}, ...]



classifier = NaiveBayesClassifier(vocabulary, word_counts_per_class, class_counts)

# for i in range(len(test_data)):
#     predicted_class = classifier.predict(test_data[i]['text'])
#     print(f"Document: {i} sample is predicted as '{predicted_class}'")

accuracy = calculate_accuracy(classifier, test_data)
print(f"Accuracy: {accuracy * 100:.2f}%")


Document sample is predicted as 'Film' (Actual: 'Film')
Document sample is predicted as 'Film' (Actual: 'Film')
Document sample is predicted as 'Film' (Actual: 'Film')
Document sample is predicted as 'Film' (Actual: 'Film')
Document sample is predicted as 'Film' (Actual: 'Film')
Document sample is predicted as 'Politician' (Actual: 'Film')
Document sample is predicted as 'Film' (Actual: 'Film')
Document sample is predicted as 'Film' (Actual: 'Film')
Document sample is predicted as 'Film' (Actual: 'Film')
Document sample is predicted as 'Film' (Actual: 'Film')
Document sample is predicted as 'Book' (Actual: 'Book')
Document sample is predicted as 'Book' (Actual: 'Book')
Document sample is predicted as 'Politician' (Actual: 'Book')
Document sample is predicted as 'Book' (Actual: 'Book')
Document sample is predicted as 'Book' (Actual: 'Book')
Document sample is predicted as 'Book' (Actual: 'Book')
Document sample is predicted as 'Book' (Actual: 'Book')
Document sample is predicted as 'Boo